<a href="https://colab.research.google.com/github/mshitie2/main/blob/main/L_CBAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#convolutional block attention module (CBAM) with  Apply Lambda layer for rescaling the attention results, VGG16
!pip install tensorflow-addons
import os
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Conv2D, Lambda, Reshape, GlobalMaxPooling2D, Average, Multiply, Concatenate, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix

# Set the path to the directory containing the dataset
dataset_dir = '/content/main_data_croped'

# Read the CSV file
data = pd.read_csv(os.path.join(dataset_dir, '/content/drive/MyDrive/computer_vision/tfti2.csv'), usecols=["key", "class"])

# Convert the 'class' column to string
data['class'] = data['class'].astype(str)

# Filter the data to include only classes 1 and 2
data = data[data['class'].isin(['1', '2', '3'])]

# Split the data into training, validation, and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=0)

# Print the number of samples in each set
print('Number of train samples:', train_data.shape[0])
print('Number of valid samples:', valid_data.shape[0])
print('Number of test samples:', test_data.shape[0])

# Preprocess data
train_data["key"] = train_data["key"].apply(lambda x: x + ".jpg")
valid_data["key"] = valid_data["key"].apply(lambda x: x + ".jpg")
test_data["key"] = test_data["key"].apply(lambda x: x + ".jpg")

BATCH_SIZE = 16
HEIGHT = 224
WIDTH = 224
N_CLASSES = 3

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=dataset_dir,
    x_col="key",
    y_col="class",
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    target_size=(HEIGHT, WIDTH),
    seed=0
)

valid_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_data,
    directory=dataset_dir,
    x_col="key",
    y_col="class",
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    target_size=(HEIGHT, WIDTH),
    seed=0
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=dataset_dir,
    x_col="key",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
    target_size=(HEIGHT, WIDTH),
    seed=0
)

# Load the VGG16 model
vgg_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(HEIGHT, WIDTH, 3)
)

# Channel attention module
x = vgg_model.output
avg_pool = GlobalAveragePooling2D()(x)
max_pool = GlobalMaxPooling2D()(x)
avg_pool = Dense(512)(avg_pool)
max_pool = Dense(512)(max_pool)
avg_pool = Activation('relu')(avg_pool)
max_pool = Activation('relu')(max_pool)
channel_attention = Average()([avg_pool, max_pool])
channel_attention = Dense(512)(channel_attention)
channel_attention = Activation('sigmoid')(channel_attention)
channel_attention = Reshape((1, 1, 512))(channel_attention)
channel_attention = Lambda(lambda x: x / tf.reduce_max(x))(channel_attention)
x = Multiply()([x, channel_attention])

# Spatial attention module
avg_pool = Lambda(lambda x: tf.keras.backend.mean(x, axis=3, keepdims=True))(x)
max_pool = Lambda(lambda x: tf.keras.backend.max(x, axis=3, keepdims=True))(x)
concat = Concatenate(axis=3)([avg_pool, max_pool])
spatial_attention = Conv2D(filters=1, kernel_size=7, padding='same', activation='sigmoid')(concat)
#spatial_attention = Lambda(lambda x: x / tf.reduce_max(x))(spatial_attention)
x = Multiply()([x, spatial_attention])

# Build the model architecture
input_tensor = Input(shape=(HEIGHT, WIDTH, 3))
x = vgg_model(input_tensor)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout regularization
x = Dense(N_CLASSES, activation='softmax', kernel_regularizer=l2(0.01), name='output')(x)
model_vgg3 = Model(inputs=input_tensor, outputs=x)


# Compile the model
model_vgg3.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy', # binary_crossentropy for two class
    metrics=['accuracy']
)

# Learning Rate Scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=1e-7)

# Fine-tune the model
EPOCHS = 50
history_model3 = model_vgg3.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator,
    callbacks=[reduce_lr],
    verbose=2
)

# Evaluate the model on the test set
test_loss, test_acc = model_vgg3.evaluate(
    test_generator,
    verbose=0
)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)